In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchtext
from torchtext.legacy import datasets

import numpy as np
import time
import os
import string
import random

In [2]:
TEXT = torchtext.legacy.data.Field(lower=True,  fix_length=200, batch_first=False)
LABEL = torchtext.legacy.data.Field(sequential=False)

In [3]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


d:\GitHub\time-series-forecasting\RNN-IMDB_dataset\.data\imdb\aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:11<00:00, 7.22MB/s]
